# Selectors

## Introduction

A selector chooses from a set of discrete options by evaluating
each of their past performances. Those use a multi-armed bandit
technique to determine which option to select next.

The selectors are intended to be used in combination with `tuners` in order to find
out and decide which model seems to get the best results once it is properly fine tuned.

## Usage examlpe

In order to use the selector we will create a ``Tuner`` instance for each model that we want to try out, as well as the ``Selector`` instance.

In this example we will use the [iris dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html)
and tune two estimators:
- [RandomForestClassifier]()
- [SGDC]()


We will start by importing our dataset, the estimators, the
train-test-splitter (we will also import a `trange` object just to
preaty print the progress bar and we will suppress the warnings):

In [1]:
import warnings

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from tqdm.auto import trange

warnings.filterwarnings('ignore')

Now we will load the dataset, split it in to train and test and
create a dictionary with the models.

In [2]:
dataset = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    dataset.data, dataset.target, test_size=0.3, random_state=0)

models = {
    'RF': RandomForestClassifier,
    'SGDC': SGDClassifier
}


Following we will import a `UCB1` selector from **BTB** and also
we will import some hyperparameters,
[Tunable](https://github.com/HDI-Project/BTB/tree/master/tutorials/01_Tuners.ipynb)
and [Tuner](https://github.com/HDI-Project/BTB/tree/master/tutorials/02_Tuners.ipynb). Then
we will create one `Tunable` object for each `model` and one `Tuner`
for each `Tunable`.


In [3]:
from btb.selection import UCB1
from btb.tuning.hyperparams import FloatHyperParam, IntHyperParam
from btb.tuning.tunable import Tunable
from btb.tuning.tuners import GPTuner

rf_hyperparams = {
    'n_estimators': IntHyperParam(min=10, max=500),
    'max_depth': IntHyperParam(min=3, max=200)
}
rf_tunable = Tunable(rf_hyperparams)

sgdc_hyperparams = {
    'max_iter': IntHyperParam(min=1, max=5000, default=1000),
    'tol': FloatHyperParam(min=1e-3, max=1, default=1e-3),
}

sgdc_tunable = Tunable(sgdc_hyperparams)
tuners = {
    'RF': GPTuner(rf_tunable),
    'SGDC': GPTuner(sgdc_tunable)
}

And finally, we create a `selector` instance with the keys that we
used for our models (estimators).

In [4]:
selector = UCB1(['RF', 'SGDC'])

Then we perform the following steps in a loop.

1. Pass all the obtained scores to the selector and let it decide which model to test.

In [5]:
next_choice = selector.select({
    'RF': tuners['RF'].scores,
    'SGDC': tuners['SGDC'].scores
})
next_choice

'RF'

2. Obtain a new set of parameters from the indicated tuner and create a model instance.

In [6]:
parameters = tuners[next_choice].propose()
model = models[next_choice](**parameters)

3. Evaluate the score of the new model instance and pass it back to the tuner

In [7]:
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
score

0.9777777777777777

In [8]:
tuners[next_choice].record(parameters, score)

Here, we will create a loop to go over for 100 iterations
and save the best score, model and parameters.

In [9]:
best_score = 0

for i in trange(100):
    next_choice = selector.select({
        'RF': tuners['RF'].scores,
        'SGDC': tuners['SGDC'].scores
    })
    parameters = tuners[next_choice].propose()
    model = models[next_choice](**parameters)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    tuners[next_choice].record(parameters, score)
    
    if score > best_score:
        best_score = score
        best_model = next_choice
        best_params = parameters

In [10]:
print(best_model, best_score, best_params)

RF 0.9777777777777777 {'n_estimators': 158, 'max_depth': 85}
